## Install Sentinel-Hub

In [ ]:
pip install sentinelhub --upgrade

     |████████████████████████████████| 194kB 4.3MB/s 
     |████████████████████████████████| 133kB 31.9MB/s 
     |████████████████████████████████| 7.2MB 45.1MB/s 
     |████████████████████████████████| 6.5MB 44.3MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 71kB 5.9MB/s 
  Created wheel for sentinelhub: filename=sentinelhub-3.1.0-cp36-none-any.whl size=196404 sha256=0ae8bdef15533db3aa4fa10e38c9f701e59516b3cb000ddb6675778beb207ae5
  Stored in directory: /root/.cache/pip/wheels/90/6c/df/be1525ead52f5c2ce49f0880e73587723bf25caac7295f9db2
  Created wheel for utm: filename=utm-0.7.0-cp36-none-any.whl size=6094 sha256=456de1d2f3a98f00266d62b474e3bae9dca6bbb83c70305f31f4fd8ca903f8fc
  Stored in directory: /root/.cache/pip/wheels/d3/a0/c4/93c7535fe705e70d2376505fd2683f91524a50a944a15212d9
Successfully built sentinelhub utm
ERROR: botocore 1.19.55 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


### Put the path to the folder where we want to save the files from Sentinelhub

In [ ]:
#Uncomment depending on the satellite 
rootPath="/content/gdrive/My Drive/WORK/C&T/SENTINEL-HUB/test_dir/SENTINEL_L2A/"


In [ ]:
from sentinelhub import SHConfig


INSTANCE_ID = '82f77578-eb9a-4909-9a28-30f5279ba6a3'  # In case you put instance ID into configuration file you can leave this unchanged

if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None

#### Imports 

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### Import the Libraries

In [ ]:
# Please, install all the neccesary libs.
import datetime
import numpy as np

import matplotlib.pyplot as plt
import os 
import json 

In [ ]:
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox, DataCollection

In [ ]:
def plot_image(image, factor=1):
    """
    Utility function for plotting RGB images.
    """
    fig = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))
    
    if np.issubdtype(image.dtype, np.floating):
        plt.imshow(np.minimum(image * factor, 1))
    else:
        plt.imshow(image)

### Setting parameters to rename the folder

In [ ]:
satellite='SentinelL2A'
campo1='campo1'

#### Setting area of interest

In [ ]:
talca_coords_wgs84 = [-71.817933, -35.452880, -71.807166, -35.446550] #TALCA coordinates 1st PART (Parte izquierda del terreno)
#FORMAT(LONGITUDE1, LATITUDE1, LONGITUDE2, LATITUDE2) From Google Earth they are lower left and upper rigth corners 

All requests require bounding box to be given as an instance of `sentinelhub.geometry.BBox` with corresponding Coordinate Reference System (`sentinelhub.geometry.CRS`). In our case it is in _WGS84_ and we can use the predefined _WGS84_ coordinate reference system from `sentinelhub.geometry.CRS`.

In [ ]:
talca_bbox = BBox(bbox=talca_coords_wgs84, crs=CRS.WGS84)

In [ ]:
wms_true_color_request = WmsRequest(
    data_collection=DataCollection.SENTINEL2_L2A,
    data_folder=rootPath,
    layer='TRUE-COLOR-S2-L2A',
    bbox=talca_bbox,
    #time='2017-12-10',
    time=('2017-01-01','2020-12-31'),
    #width=512,
    #height=856,
    width=600,
    #height=600,
    image_format=MimeType.TIFF,
    time_difference=datetime.timedelta(hours=2),
    config=config
)

In [ ]:
#Save data
%%time
wms_true_color_request.save_data()


CPU times: user 3.77 s, sys: 468 ms, total: 4.24 s
Wall time: 31.3 s


## PRUEBA DE NUMERO DE IMAGENES Q DESCARGA

In [ ]:
wms_bands_img_from_disk = wms_true_color_request .get_data()
print('These %d images were taken on the following dates:' % len(wms_bands_img_from_disk))
for index, date in enumerate(wms_true_color_request.get_dates()):
    print(' - image %d was taken on %s' % (index, date))

These 230 images were taken on the following dates:
 - image 0 was taken on 2017-01-19 14:45:47
 - image 1 was taken on 2017-01-29 14:44:58
 - image 2 was taken on 2017-02-18 14:51:50
 - image 3 was taken on 2017-02-28 14:45:05
 - image 4 was taken on 2017-03-20 14:51:59
 - image 5 was taken on 2017-04-09 14:49:33
 - image 6 was taken on 2017-04-29 14:49:45
 - image 7 was taken on 2017-05-19 14:49:48
 - image 8 was taken on 2017-06-08 14:49:11
 - image 9 was taken on 2017-06-28 14:49:05
 - image 10 was taken on 2017-07-13 14:49:43
 - image 11 was taken on 2017-07-18 14:49:10
 - image 12 was taken on 2017-08-02 14:49:42
 - image 13 was taken on 2017-08-27 14:49:09
 - image 14 was taken on 2017-09-11 14:50:07
 - image 15 was taken on 2017-09-16 14:51:18
 - image 16 was taken on 2017-10-01 14:45:37
 - image 17 was taken on 2017-10-06 14:48:51
 - image 18 was taken on 2017-10-21 14:49:45
 - image 19 was taken on 2017-10-26 14:41:12
 - image 20 was taken on 2017-11-10 14:40:29
 - image 21 w

In [ ]:
#Getting the list of all saved folders.
folders1=os.listdir(wms_true_color_request.data_folder)
print(folders1)

['01826a4182f0dd0ad5f2b227da16d7b5', '5974a08ed9aa636328805a58385a2421', '392b536057f6ca7277633967c50cc000', '102e259a81d8e365c21adcdb5715db8e', '0c2d7c6a48939f777d0e3175c10c8c87', 'b05a468a6a7fac7ee4a16de8570594b1', 'd47167168722d2e95207d046583b168f', 'c99bdc6e85fc92fcd49629cbad4317a0', 'a5babdb5a43c76dd26c509284170fdea', '1c8d798f551f4d29a9c7001ed42707ef', 'c11e200cb307a9e886a0ac9725bfc774', '0ba25d5f7b775d5ae0b357d6c5e3f0af', '687fb010c45d11d542398f9c79558abb', '7819fb84ef6295d63b935d871bab3381', '3e5b2473ca6c3e89f2cafbcae3058cff', 'fc8267837687d037e5126580bbc8ecbe', '997d3eaa3c1ec9d92e7adcecd8395260', '5dfdbd103967a5b330c090b5ed90e3cb', '4d167357a63b5ef29a57c6dc2a8f6353', '7924795a452f14cf8775838f92a33a94', '3153aae7362c8ab9f4df492574c3c15a', '863b3cab5a2dc919986f0955dba6fe36', '98c48de9399c31b51793cc1136c561af', 'c821360745506e9c236f9a8986f03882', '7f16d54dcd982d0f4d5a02a704a6bc28', 'ff3823b95f5aa06aba23def884da46fa', '7338e1673e92aa7821a865201b14546c', 'ce2ff77e67569b6a19735d4840

### Setting parameters to rename the folder

In [ ]:
satellite='SentinelL2A'
campo2='campo2'

#### Setting area of interest

In [ ]:
talca_coords_wgs84 = [-71.800228, -35.457577, -71.786453, -35.449821] #TALCA coordinates 2nd PART (Parte derecha del terreno)
#FORMAT(LONGITUDE1, LATITUDE1, LONGITUDE2, LATITUDE2) From Google Earth they are lower left and upper rigth corners 

All requests require bounding box to be given as an instance of `sentinelhub.geometry.BBox` with corresponding Coordinate Reference System (`sentinelhub.geometry.CRS`). In our case it is in _WGS84_ and we can use the predefined _WGS84_ coordinate reference system from `sentinelhub.geometry.CRS`.

In [ ]:
talca_bbox = BBox(bbox=talca_coords_wgs84, crs=CRS.WGS84)

In [ ]:
wms_true_color_request = WmsRequest(
    data_collection=DataCollection.SENTINEL2_L2A,
    data_folder=rootPath,
    layer='TRUE-COLOR-S2-L2A',
    bbox=talca_bbox,
    #time='2017-12-10',
    time=('2017-01-01','2020-12-31'),
    #width=512,
    #height=856,
    width=600,
    #height=600,
    image_format=MimeType.TIFF,
    time_difference=datetime.timedelta(hours=2),
    config=config
)

In [ ]:
#Save data
%%time
wms_true_color_request.save_data()


CPU times: user 3.85 s, sys: 479 ms, total: 4.33 s
Wall time: 28.1 s


## PRUEBA DE NUMERO DE IMAGENES Q DESCARGA

In [ ]:
wms_bands_img_from_disk = wms_true_color_request .get_data()
print('These %d images were taken on the following dates:' % len(wms_bands_img_from_disk))
for index, date in enumerate(wms_true_color_request.get_dates()):
    print(' - image %d was taken on %s' % (index, date))

These 230 images were taken on the following dates:
 - image 0 was taken on 2017-01-19 14:45:47
 - image 1 was taken on 2017-01-29 14:44:58
 - image 2 was taken on 2017-02-18 14:51:50
 - image 3 was taken on 2017-02-28 14:45:05
 - image 4 was taken on 2017-03-20 14:51:59
 - image 5 was taken on 2017-04-09 14:49:33
 - image 6 was taken on 2017-04-29 14:49:45
 - image 7 was taken on 2017-05-19 14:49:48
 - image 8 was taken on 2017-06-08 14:49:11
 - image 9 was taken on 2017-06-28 14:49:05
 - image 10 was taken on 2017-07-13 14:49:43
 - image 11 was taken on 2017-07-18 14:49:10
 - image 12 was taken on 2017-08-02 14:49:42
 - image 13 was taken on 2017-08-27 14:49:09
 - image 14 was taken on 2017-09-11 14:50:07
 - image 15 was taken on 2017-09-16 14:51:18
 - image 16 was taken on 2017-10-01 14:45:37
 - image 17 was taken on 2017-10-06 14:48:51
 - image 18 was taken on 2017-10-21 14:49:45
 - image 19 was taken on 2017-10-26 14:41:12
 - image 20 was taken on 2017-11-10 14:40:29
 - image 21 w

In [ ]:
#Getting the list of all saved folders.
folders2=os.listdir(wms_true_color_request.data_folder)

for i in folders1:
  for j in folders2:
    if i == j:
      folders2.remove(j)
print(folders1)
print(folders2)

['01826a4182f0dd0ad5f2b227da16d7b5', '5974a08ed9aa636328805a58385a2421', '392b536057f6ca7277633967c50cc000', '102e259a81d8e365c21adcdb5715db8e', '0c2d7c6a48939f777d0e3175c10c8c87', 'b05a468a6a7fac7ee4a16de8570594b1', 'd47167168722d2e95207d046583b168f', 'c99bdc6e85fc92fcd49629cbad4317a0', 'a5babdb5a43c76dd26c509284170fdea', '1c8d798f551f4d29a9c7001ed42707ef', 'c11e200cb307a9e886a0ac9725bfc774', '0ba25d5f7b775d5ae0b357d6c5e3f0af', '687fb010c45d11d542398f9c79558abb', '7819fb84ef6295d63b935d871bab3381', '3e5b2473ca6c3e89f2cafbcae3058cff', 'fc8267837687d037e5126580bbc8ecbe', '997d3eaa3c1ec9d92e7adcecd8395260', '5dfdbd103967a5b330c090b5ed90e3cb', '4d167357a63b5ef29a57c6dc2a8f6353', '7924795a452f14cf8775838f92a33a94', '3153aae7362c8ab9f4df492574c3c15a', '863b3cab5a2dc919986f0955dba6fe36', '98c48de9399c31b51793cc1136c561af', 'c821360745506e9c236f9a8986f03882', '7f16d54dcd982d0f4d5a02a704a6bc28', 'ff3823b95f5aa06aba23def884da46fa', '7338e1673e92aa7821a865201b14546c', 'ce2ff77e67569b6a19735d4840

### Etiquetar las imágenes con el # de campo y fecha respectiva.

In [ ]:
for folder in folders1:
  with open(rootPath+folder+"/request.json") as file: #Define the path to the folder and to the json file
    dic=json.load(file)
  dic1=dic.copy() #For the anotations
  url=dic1['url']  #key url where is the date
  date=[]
  date_str=""
  for i in range(len(url)):
      if url[i]=='T' and url[i+1]=='I' and url[i+2]=='M' and url[i+3]=='E':
          for j in range(10):
              date.append(url[i+5+j])
  date_str=date_str.join(date)
  date_str1=campo1+"_"+date_str
  image_path=rootPath+folder+"/response.tiff"
  new_name=rootPath+folder+"/"+date_str1+".tiff"
  os.rename(image_path,new_name)

  #Change the name of the folder
  folder_name=rootPath+folder
  new_folder_name=rootPath+campo1+"_"+satellite+"_"+date_str
  os.rename(folder_name,new_folder_name)

   


### Etiquetar las imágenes con el # de campo y fecha respectiva.

In [ ]:
for folder in folders2:
  with open(rootPath+folder+"/request.json") as file: #Define the path to the folder and to the json file
    dic=json.load(file)
  dic1=dic.copy() #For the anotations
  url=dic1['url']  #key url where is the date
  date=[]
  date_str=""
  for i in range(len(url)):
      if url[i]=='T' and url[i+1]=='I' and url[i+2]=='M' and url[i+3]=='E':
          for j in range(10):
              date.append(url[i+5+j])
  date_str=date_str.join(date)
  date_str1=campo2+"_"+date_str
  image_path=rootPath+folder+"/response.tiff"
  new_name=rootPath+folder+"/"+date_str1+".tiff"
  os.rename(image_path,new_name)

  #Change the name of the folder
  folder_name=rootPath+folder
  new_folder_name=rootPath+campo2+"_"+satellite+"_"+date_str
  os.rename(folder_name,new_folder_name)

   
